# Tarea 7: Reconocimiento Facial

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import json

import mlflow
import mlflow.pytorch
import optuna

plt.style.use('default')
%matplotlib inline

print(f"PyTorch: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
print(f"GPUs detectadas: {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

dispositivo = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
usar_multi_gpu = torch.cuda.device_count() > 1

PyTorch: 2.1.2
CUDA disponible: True
GPUs detectadas: 2
GPU 0: Tesla P100-PCIE-16GB
GPU 1: Tesla P100-PCIE-16GB


In [30]:
class Configuracion:
    DIR_BASE = Path("/home2/DSMaster/mgarcia/Tarea 7")
    DIR_DATOS = DIR_BASE / "datos"
    DIR_CELEBA = DIR_DATOS / "celeba"
    IMAGENES_CELEBA = DIR_CELEBA / "img_align_celeba"
    ARCHIVO_ATRIBUTOS_CELEBA = DIR_CELEBA / "list_attr_celeba.txt"
    DIR_MI_ROSTRO = DIR_DATOS / "mi_rostro"
    DIR_MODELOS = DIR_BASE / "modelos"
    DIR_RESULTADOS = DIR_BASE / "resultados"
    DIR_MLRUNS = DIR_BASE / "mlruns"
    
    DIR_MODELOS.mkdir(exist_ok=True)
    DIR_RESULTADOS.mkdir(exist_ok=True)
    DIR_MLRUNS.mkdir(exist_ok=True)
    
    TAMANO_IMG = 128
    NUM_WORKERS = 8
    
    LISTA_ATRIBUTOS_CELEBA = [
        'Attractive', 'Male', 'Smiling', 'Wearing_Lipstick',
        'Young', 'Eyeglasses', 'Black_Hair', 'Blond_Hair', 'Brown_Hair']
    
    NUM_ATRIBUTOS_CELEBA = len(LISTA_ATRIBUTOS_CELEBA)
    
    TRIALS_PREENTRENAMIENTO = 5
    TRIALS_TRANSFER = 10
    TRIALS_FINETUNE = 5
    
    MLFLOW_URI = f"file:{DIR_MLRUNS}"
    NOMBRE_EXPERIMENTO = "reconocimiento_facial_optuna"
    SEMILLA = 42

torch.manual_seed(Configuracion.SEMILLA)
np.random.seed(Configuracion.SEMILLA)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(Configuracion.SEMILLA)

mlflow.set_tracking_uri(Configuracion.MLFLOW_URI)
mlflow.set_experiment(Configuracion.NOMBRE_EXPERIMENTO)

print("Configuracion lista")

Configuracion lista


In [31]:
class DatasetCelebA(Dataset):
    def __init__(self, dir_imagenes, archivo_atributos, atributos, transformacion=None, limite=None):
        self.dir_imagenes = Path(dir_imagenes)
        self.transformacion = transformacion
        self.atributos = atributos
        df = pd.read_csv(archivo_atributos)
        self.datos = df[['image_id'] + atributos].copy()
        for attr in atributos:
            self.datos[attr] = (self.datos[attr] + 1) // 2
        if limite:
            self.datos = self.datos.head(limite)
        print(f"Dataset CelebA: {len(self.datos)} imagenes")
    
    def __len__(self):
        return len(self.datos)
    
    def __getitem__(self, idx):
        fila = self.datos.iloc[idx]
        ruta_img = self.dir_imagenes / fila['image_id']
        img = Image.open(ruta_img)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        if self.transformacion:
            img = np.array(img, dtype=np.uint8)
            img = Image.fromarray(img)
            img = self.transformacion(img)
        etiquetas = torch.tensor(fila[self.atributos].values.astype(np.float32))
        return img, etiquetas


class DatasetReconocimientoFacial(Dataset):
    def __init__(self, dir_positivas, dir_negativas, transformacion=None, muestras_negativas=None):
        self.transformacion = transformacion
        self.muestras = []
        dir_positivas = Path(dir_positivas)
        for ext in ['*.jpg', '*.png']:
            for ruta_img in dir_positivas.glob(ext):
                self.muestras.append((str(ruta_img), 1))
        num_positivas = len(self.muestras)
        print(f"Imagenes positivas: {num_positivas}")
        dir_negativas = Path(dir_negativas)
        imagenes_negativas = list(dir_negativas.glob('*.jpg'))
        if muestras_negativas:
            imagenes_negativas = imagenes_negativas[:muestras_negativas]
        else:
            imagenes_negativas = imagenes_negativas[:num_positivas * 2]
        for ruta_img in imagenes_negativas:
            self.muestras.append((str(ruta_img), 0))
        print(f"Imagenes negativas: {len(imagenes_negativas)}")
    
    def __len__(self):
        return len(self.muestras)
    
    def __getitem__(self, idx):
        ruta_img, etiqueta = self.muestras[idx]
        img = Image.open(ruta_img).convert('RGB')
        if self.transformacion:
            img = self.transformacion(img)
        return img, torch.tensor(etiqueta, dtype=torch.float32)


def obtener_transformaciones(tamano_img, aumentar=True):
    if aumentar:
        return transforms.Compose([
            transforms.Resize((tamano_img, tamano_img)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(20),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
    return transforms.Compose([
        transforms.Resize((tamano_img, tamano_img)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

print("Datasets listos")

Datasets listos


In [32]:
class ModeloCelebA(nn.Module):
    def __init__(self, num_atributos=9, fc_oculto=1024, dropout=0.5):
        super().__init__()
        canales_conv = [64, 128, 256, 512, 512]
        capas = []
        canales_entrada = 3
        for canales_salida in canales_conv:
            capas.extend([
                nn.Conv2d(canales_entrada, canales_salida, 3, padding=1),
                nn.BatchNorm2d(canales_salida),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, 2)
            ])
            canales_entrada = canales_salida
        self.caracteristicas = nn.Sequential(*capas)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.clasificador = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, fc_oculto),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(fc_oculto, num_atributos))
    
    def forward(self, x):
        return self.clasificador(self.gap(self.caracteristicas(x)))


class ModeloReconocimientoFacial(nn.Module):
    def __init__(self, modelo_preentrenado, fc_oculto=256, dropout=0.3):
        super().__init__()
        self.caracteristicas = modelo_preentrenado.caracteristicas
        self.gap = modelo_preentrenado.gap
        self.clasificador = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, fc_oculto),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(fc_oculto, 1))
    
        for param in self.caracteristicas.parameters():
            param.requires_grad = False
        for param in self.gap.parameters():
            param.requires_grad = False
    
    def forward(self, x):
        return self.clasificador(self.gap(self.caracteristicas(x))).squeeze(1)
    
    def descongelar_capas(self, num_capas=6):
        for capa in list(self.caracteristicas.children())[-num_capas:]:
            for param in capa.parameters():
                param.requires_grad = True

print("Modelos listos")

Modelos listos


In [33]:
def entrenar_epoca(modelo, cargador, criterio, optimizador, dispositivo, multietiqueta=False):
    modelo.train()
    perdida_total = 0.0
    correctos = 0
    total = 0
    for entradas, etiquetas in cargador:
        entradas, etiquetas = entradas.to(dispositivo), etiquetas.to(dispositivo)
        optimizador.zero_grad()
        salidas = modelo(entradas)
        perdida = criterio(salidas, etiquetas)
        perdida.backward()
        optimizador.step()
        perdida_total += perdida.item() * entradas.size(0)
        predicciones = (torch.sigmoid(salidas) > 0.5).float()
        if multietiqueta:
            correctos += (predicciones == etiquetas).sum().item()
            total += etiquetas.numel()
        else:
            correctos += (predicciones == etiquetas).sum().item()
            total += etiquetas.size(0)
    return perdida_total / len(cargador.dataset), correctos / total


def validar_epoca(modelo, cargador, criterio, dispositivo, multietiqueta=False):
    modelo.eval()
    perdida_total = 0.0
    correctos = 0
    total = 0
    with torch.no_grad():
        for entradas, etiquetas in cargador:
            entradas, etiquetas = entradas.to(dispositivo), etiquetas.to(dispositivo)
            salidas = modelo(entradas)
            perdida = criterio(salidas, etiquetas)
            perdida_total += perdida.item() * entradas.size(0)
            predicciones = (torch.sigmoid(salidas) > 0.5).float()
            if multietiqueta:
                correctos += (predicciones == etiquetas).sum().item()
                total += etiquetas.numel()
            else:
                correctos += (predicciones == etiquetas).sum().item()
                total += etiquetas.size(0)
    return perdida_total / len(cargador.dataset), correctos / total

print("Funciones de entrenamiento listas")

Funciones de entrenamiento listas


In [34]:
print("Cargando dataset CelebA")

dataset_completo = DatasetCelebA(
    Configuracion.IMAGENES_CELEBA,
    Configuracion.ARCHIVO_ATRIBUTOS_CELEBA,
    Configuracion.LISTA_ATRIBUTOS_CELEBA,
    transformacion=None,
    limite=None)

tamano_entrenamiento = int(0.9 * len(dataset_completo))
tamano_validacion = len(dataset_completo) - tamano_entrenamiento
indices_entrenamiento, indices_validacion = torch.utils.data.random_split(
    range(len(dataset_completo)), [tamano_entrenamiento, tamano_validacion])

class SubconjuntoTransformado(Dataset):
    def __init__(self, dataset, indices, transformacion):
        self.dataset = dataset
        self.indices = indices
        self.transformacion = transformacion
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        idx_real = self.indices[idx]
        fila = self.dataset.datos.iloc[idx_real]
        ruta_img = self.dataset.dir_imagenes / fila['image_id']
        img = Image.open(ruta_img)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        if self.transformacion:
            img = np.array(img, dtype=np.uint8)
            img = Image.fromarray(img)
            img = self.transformacion(img)
        etiquetas = torch.tensor(fila[self.dataset.atributos].values.astype(np.float32))
        return img, etiquetas

print(f"Entrenamiento: {len(indices_entrenamiento)}")
print(f"Validacion: {len(indices_validacion)}")

Cargando dataset CelebA
Dataset CelebA: 202599 imagenes
Entrenamiento: 182339
Validacion: 20260


In [35]:
print("Optimizando hiperparametros de pre-entrenamiento")
print(f"Trials: {Configuracion.TRIALS_PREENTRENAMIENTO}")

def objetivo_preentrenamiento(trial):
    tasa_aprendizaje = trial.suggest_loguniform('tasa_aprendizaje', 1e-4, 1e-2)
    decaimiento_peso = trial.suggest_loguniform('decaimiento_peso', 1e-6, 1e-3)
    dropout = trial.suggest_uniform('dropout', 0.3, 0.7)
    fc_oculto = trial.suggest_categorical('fc_oculto', [512, 1024, 2048])
    tamano_lote = trial.suggest_categorical('tamano_lote', [32, 64, 128])
    
    transformacion_entrenamiento = obtener_transformaciones(Configuracion.TAMANO_IMG, aumentar=True)
    transformacion_validacion = obtener_transformaciones(Configuracion.TAMANO_IMG, aumentar=False)
    
    dataset_entrenamiento = SubconjuntoTransformado(dataset_completo, indices_entrenamiento.indices, transformacion_entrenamiento)
    dataset_validacion = SubconjuntoTransformado(dataset_completo, indices_validacion.indices, transformacion_validacion)
    
    cargador_entrenamiento = DataLoader(dataset_entrenamiento, batch_size=tamano_lote, shuffle=True, 
                            num_workers=Configuracion.NUM_WORKERS, pin_memory=True)
    cargador_validacion = DataLoader(dataset_validacion, batch_size=tamano_lote, shuffle=False,
                          num_workers=Configuracion.NUM_WORKERS, pin_memory=True)
    
    modelo = ModeloCelebA(num_atributos=Configuracion.NUM_ATRIBUTOS_CELEBA, 
                       fc_oculto=fc_oculto, dropout=dropout)
    if usar_multi_gpu:
        modelo = nn.DataParallel(modelo)
    modelo = modelo.to(dispositivo)
    
    criterio = nn.BCEWithLogitsLoss()
    optimizador = optim.Adam(modelo.parameters(), lr=tasa_aprendizaje, weight_decay=decaimiento_peso)
    
    mejor_accuracy_val = 0.0
    for epoca in range(10):
        perdida_entrenamiento, accuracy_entrenamiento = entrenar_epoca(modelo, cargador_entrenamiento, criterio, optimizador, dispositivo, True)
        perdida_validacion, accuracy_validacion = validar_epoca(modelo, cargador_validacion, criterio, dispositivo, True)
        if accuracy_validacion > mejor_accuracy_val:
            mejor_accuracy_val = accuracy_validacion
        trial.report(accuracy_validacion, epoca)
        if trial.should_prune():
            raise optuna.TrialPruned()
    return mejor_accuracy_val

estudio_preentrenamiento = optuna.create_study(
    direction='maximize',
    study_name='preentrenamiento_celeba',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=3))

estudio_preentrenamiento.optimize(objetivo_preentrenamiento, n_trials=Configuracion.TRIALS_PREENTRENAMIENTO, show_progress_bar=True)

print(f"Mejor accuracy: {estudio_preentrenamiento.best_value:.4f}")
print(f"Mejores parametros: {estudio_preentrenamiento.best_params}")

[I 2025-12-02 01:58:29,625] A new study created in memory with name: preentrenamiento_celeba


Optimizando hiperparametros de pre-entrenamiento
Trials: 5


  0%|          | 0/5 [00:00<?, ?it/s]

/tmp/ipykernel_1194169/1499883838.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  tasa_aprendizaje = trial.suggest_loguniform('tasa_aprendizaje', 1e-4, 1e-2)
/tmp/ipykernel_1194169/1499883838.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  decaimiento_peso = trial.suggest_loguniform('decaimiento_peso', 1e-6, 1e-3)
/tmp/ipykernel_1194169/1499883838.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout = trial.suggest_uniform('dropout', 0.3, 0.7)


[I 2025-12-02 02:23:51,024] Trial 0 finished with value: 0.9140890643852144 and parameters: {'tasa_aprendizaje': 0.001211655178066541, 'decaimiento_peso': 3.8792665915804016e-05, 'dropout': 0.4695498190853449, 'fc_oculto': 512, 'tamano_lote': 64}. Best is trial 0 with value: 0.9140890643852144.
[I 2025-12-02 03:01:42,909] Trial 1 finished with value: 0.9203082154217396 and parameters: {'tasa_aprendizaje': 0.00033720802263697533, 'decaimiento_peso': 1.023229747601771e-06, 'dropout': 0.49197381781766425, 'fc_oculto': 2048, 'tamano_lote': 32}. Best is trial 1 with value: 0.9203082154217396.
[I 2025-12-02 03:38:47,051] Trial 2 finished with value: 0.9202040144784469 and parameters: {'tasa_aprendizaje': 0.00017646555049643132, 'decaimiento_peso': 1.7359348676460013e-05, 'dropout': 0.5570777164689342, 'fc_oculto': 2048, 'tamano_lote': 32}. Best is trial 1 with value: 0.9203082154217396.
[I 2025-12-02 04:14:34,333] Trial 3 finished with value: 0.9149994515739827 and parameters: {'tasa_aprendi

In [36]:
print("Entrenando modelo pre-entrenado final")

mejores_params = estudio_preentrenamiento.best_params
transformacion_entrenamiento = obtener_transformaciones(Configuracion.TAMANO_IMG, aumentar=True)
transformacion_validacion = obtener_transformaciones(Configuracion.TAMANO_IMG, aumentar=False)

dataset_entrenamiento = SubconjuntoTransformado(dataset_completo, indices_entrenamiento.indices, transformacion_entrenamiento)
dataset_validacion = SubconjuntoTransformado(dataset_completo, indices_validacion.indices, transformacion_validacion)

cargador_entrenamiento = DataLoader(dataset_entrenamiento, batch_size=mejores_params['tamano_lote'], 
                        shuffle=True, num_workers=Configuracion.NUM_WORKERS, pin_memory=True)
cargador_validacion = DataLoader(dataset_validacion, batch_size=mejores_params['tamano_lote'],
                      shuffle=False, num_workers=Configuracion.NUM_WORKERS, pin_memory=True)

modelo_celeba = ModeloCelebA(
    num_atributos=Configuracion.NUM_ATRIBUTOS_CELEBA,
    fc_oculto=mejores_params['fc_oculto'],
    dropout=mejores_params['dropout'])

if usar_multi_gpu:
    modelo_celeba = nn.DataParallel(modelo_celeba)
modelo_celeba = modelo_celeba.to(dispositivo)

criterio = nn.BCEWithLogitsLoss()
optimizador = optim.Adam(modelo_celeba.parameters(), lr=mejores_params['tasa_aprendizaje'], 
                      weight_decay=mejores_params['decaimiento_peso'])

num_epocas = 20
mejor_accuracy_val = 0.0
historial_preentrenamiento = {'perdida_entrenamiento': [], 'accuracy_entrenamiento': [], 
                              'perdida_validacion': [], 'accuracy_validacion': []}

for epoca in range(num_epocas):
    print(f"Epoca {epoca+1}/{num_epocas}")
    perdida_entrenamiento, accuracy_entrenamiento = entrenar_epoca(modelo_celeba, cargador_entrenamiento, criterio, optimizador, dispositivo, True)
    perdida_validacion, accuracy_validacion = validar_epoca(modelo_celeba, cargador_validacion, criterio, dispositivo, True)
    
    historial_preentrenamiento['perdida_entrenamiento'].append(perdida_entrenamiento)
    historial_preentrenamiento['accuracy_entrenamiento'].append(accuracy_entrenamiento)
    historial_preentrenamiento['perdida_validacion'].append(perdida_validacion)
    historial_preentrenamiento['accuracy_validacion'].append(accuracy_validacion)
    
    print(f"Entrenamiento: {perdida_entrenamiento:.4f}/{accuracy_entrenamiento:.4f}, Validacion: {perdida_validacion:.4f}/{accuracy_validacion:.4f}")
    
    if accuracy_validacion > mejor_accuracy_val:
        mejor_accuracy_val = accuracy_validacion
        modelo_guardar = modelo_celeba.module if usar_multi_gpu else modelo_celeba
        torch.save(modelo_guardar.state_dict(), Configuracion.DIR_MODELOS / "celeba_mejor.pth")

modelo_guardar = modelo_celeba.module if usar_multi_gpu else modelo_celeba
torch.save(modelo_guardar.state_dict(), Configuracion.DIR_MODELOS / "celeba_final.pth")
print(f"Pre-entrenamiento completado. Mejor Accuracy Val: {mejor_accuracy_val:.4f}")

Entrenando modelo pre-entrenado final
Epoca 1/20
Entrenamiento: 0.2722/0.8786, Validacion: 0.2244/0.9016
Epoca 2/20
Entrenamiento: 0.2185/0.9049, Validacion: 0.2200/0.9058
Epoca 3/20
Entrenamiento: 0.2072/0.9097, Validacion: 0.1949/0.9147
Epoca 4/20
Entrenamiento: 0.2005/0.9127, Validacion: 0.1903/0.9158
Epoca 5/20
Entrenamiento: 0.1960/0.9146, Validacion: 0.1855/0.9190
Epoca 6/20
Entrenamiento: 0.1922/0.9162, Validacion: 0.1811/0.9209
Epoca 7/20
Entrenamiento: 0.1891/0.9174, Validacion: 0.1860/0.9204
Epoca 8/20
Entrenamiento: 0.1868/0.9189, Validacion: 0.1785/0.9218
Epoca 9/20
Entrenamiento: 0.1845/0.9197, Validacion: 0.1860/0.9186
Epoca 10/20
Entrenamiento: 0.1826/0.9206, Validacion: 0.1812/0.9207
Epoca 11/20
Entrenamiento: 0.1805/0.9214, Validacion: 0.1966/0.9171
Epoca 12/20
Entrenamiento: 0.1790/0.9219, Validacion: 0.1884/0.9178
Epoca 13/20
Entrenamiento: 0.1773/0.9227, Validacion: 0.1782/0.9217
Epoca 14/20
Entrenamiento: 0.1756/0.9234, Validacion: 0.1808/0.9204
Epoca 15/20
Entrena

In [37]:
print("Cargando dataset de reconocimiento facial")

dataset_rostros_completo = DatasetReconocimientoFacial(
    Configuracion.DIR_MI_ROSTRO,
    Configuracion.IMAGENES_CELEBA,
    transformacion=None,
    muestras_negativas=200)

tamano_entrenamiento = int(0.8 * len(dataset_rostros_completo))
tamano_validacion = len(dataset_rostros_completo) - tamano_entrenamiento
indices_rostros_entrenamiento, indices_rostros_validacion = torch.utils.data.random_split(
    range(len(dataset_rostros_completo)), [tamano_entrenamiento, tamano_validacion])

class SubconjuntoRostros(Dataset):
    def __init__(self, dataset, indices, transformacion):
        self.dataset = dataset
        self.indices = indices
        self.transformacion = transformacion
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        idx_real = self.indices[idx]
        ruta_img, etiqueta = self.dataset.muestras[idx_real]
        img = Image.open(ruta_img).convert('RGB')
        if self.transformacion:
            img = self.transformacion(img)
        return img, torch.tensor(etiqueta, dtype=torch.float32)

print(f"Entrenamiento: {len(indices_rostros_entrenamiento)}")
print(f"Validacion: {len(indices_rostros_validacion)}")

Cargando dataset de reconocimiento facial
Imagenes positivas: 45
Imagenes negativas: 200
Entrenamiento: 196
Validacion: 49


In [41]:
print("Optimizando hiperparametros de transfer learning")
print(f"Trials: {Configuracion.TRIALS_TRANSFER}")

modelo_base = modelo_celeba.module if usar_multi_gpu else modelo_celeba

def objetivo_transfer(trial):
    tasa_aprendizaje = trial.suggest_loguniform('tasa_aprendizaje', 1e-5, 1e-2)
    decaimiento_peso = trial.suggest_loguniform('decaimiento_peso', 1e-6, 1e-3)
    dropout = trial.suggest_uniform('dropout', 0.2, 0.5)
    fc_oculto = trial.suggest_categorical('fc_oculto', [128, 256, 512])
    tamano_lote = trial.suggest_categorical('tamano_lote', [16, 32, 64])
    
    transformacion_entrenamiento = obtener_transformaciones(Configuracion.TAMANO_IMG, aumentar=True)
    transformacion_validacion = obtener_transformaciones(Configuracion.TAMANO_IMG, aumentar=False)
    
    dataset_entrenamiento = SubconjuntoRostros(dataset_rostros_completo, indices_rostros_entrenamiento.indices, transformacion_entrenamiento)
    dataset_validacion = SubconjuntoRostros(dataset_rostros_completo, indices_rostros_validacion.indices, transformacion_validacion)
    
    cargador_entrenamiento = DataLoader(dataset_entrenamiento, batch_size=tamano_lote, shuffle=True,
                            num_workers=Configuracion.NUM_WORKERS, pin_memory=True)
    cargador_validacion = DataLoader(dataset_validacion, batch_size=tamano_lote, shuffle=False,
                          num_workers=Configuracion.NUM_WORKERS, pin_memory=True)
    
    modelo = ModeloReconocimientoFacial(modelo_base, fc_oculto=fc_oculto, dropout=dropout)

    # Usar solo GPU 0 para optimización
    modelo = modelo.to('cuda:0')
    
    criterio = nn.BCEWithLogitsLoss()
    parametros_entrenables = [p for p in modelo.parameters() if p.requires_grad]
    optimizador = optim.Adam(parametros_entrenables, lr=tasa_aprendizaje, weight_decay=decaimiento_peso)
    
    mejor_accuracy_val = 0.0
    for epoca in range(30):
        perdida_entrenamiento, accuracy_entrenamiento = entrenar_epoca(modelo, cargador_entrenamiento, criterio, optimizador, 'cuda:0', False)
        perdida_validacion, accuracy_validacion = validar_epoca(modelo, cargador_validacion, criterio, 'cuda:0', False)
        if accuracy_validacion > mejor_accuracy_val:
            mejor_accuracy_val = accuracy_validacion
        trial.report(accuracy_validacion, epoca)
        if trial.should_prune():
            raise optuna.TrialPruned()
    return mejor_accuracy_val

estudio_transfer = optuna.create_study(
    direction='maximize',
    study_name='transfer_learning',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=5))

estudio_transfer.optimize(objetivo_transfer, n_trials=Configuracion.TRIALS_TRANSFER, show_progress_bar=True)

print(f"Mejor accuracy: {estudio_transfer.best_value:.4f}")
print(f"Mejores parametros: {estudio_transfer.best_params}")

[I 2025-12-02 11:21:49,210] A new study created in memory with name: transfer_learning


Optimizando hiperparametros de transfer learning
Trials: 10


  0%|          | 0/10 [00:00<?, ?it/s]

/tmp/ipykernel_1194169/2939482511.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  tasa_aprendizaje = trial.suggest_loguniform('tasa_aprendizaje', 1e-5, 1e-2)
/tmp/ipykernel_1194169/2939482511.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  decaimiento_peso = trial.suggest_loguniform('decaimiento_peso', 1e-6, 1e-3)
/tmp/ipykernel_1194169/2939482511.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout = trial.suggest_uniform('dropout', 0.2, 0.5)


[I 2025-12-02 11:22:18,928] Trial 0 finished with value: 1.0 and parameters: {'tasa_aprendizaje': 0.0005837162657342226, 'decaimiento_peso': 5.823618483580234e-05, 'dropout': 0.24976251670165306, 'fc_oculto': 512, 'tamano_lote': 16}. Best is trial 0 with value: 1.0.
[I 2025-12-02 11:22:51,226] Trial 1 finished with value: 0.9795918367346939 and parameters: {'tasa_aprendizaje': 0.004157868720252427, 'decaimiento_peso': 2.483335353558449e-06, 'dropout': 0.33055656180346077, 'fc_oculto': 512, 'tamano_lote': 32}. Best is trial 0 with value: 1.0.
[I 2025-12-02 11:23:19,958] Trial 2 finished with value: 1.0 and parameters: {'tasa_aprendizaje': 0.0030568323149573464, 'decaimiento_peso': 0.0005404294806991425, 'dropout': 0.20597626086770432, 'fc_oculto': 512, 'tamano_lote': 16}. Best is trial 0 with value: 1.0.
[I 2025-12-02 11:23:52,364] Trial 3 finished with value: 0.7959183673469388 and parameters: {'tasa_aprendizaje': 0.00023030466766782914, 'decaimiento_peso': 3.145864602136453e-06, 'drop

In [43]:
print("Entrenando modelo de transfer learning final")

mejores_params_transfer = estudio_transfer.best_params
transformacion_entrenamiento = obtener_transformaciones(Configuracion.TAMANO_IMG, aumentar=True)
transformacion_validacion = obtener_transformaciones(Configuracion.TAMANO_IMG, aumentar=False)

dataset_entrenamiento = SubconjuntoRostros(dataset_rostros_completo, indices_rostros_entrenamiento.indices, transformacion_entrenamiento)
dataset_validacion = SubconjuntoRostros(dataset_rostros_completo, indices_rostros_validacion.indices, transformacion_validacion)

cargador_entrenamiento = DataLoader(dataset_entrenamiento, batch_size=mejores_params_transfer['tamano_lote'],
                        shuffle=True, num_workers=Configuracion.NUM_WORKERS, pin_memory=True)
cargador_validacion = DataLoader(dataset_validacion, batch_size=mejores_params_transfer['tamano_lote'],
                      shuffle=False, num_workers=Configuracion.NUM_WORKERS, pin_memory=True)

modelo_rostros = ModeloReconocimientoFacial(
    modelo_base,
    fc_oculto=mejores_params_transfer['fc_oculto'],
    dropout=mejores_params_transfer['dropout'])

if usar_multi_gpu:
    modelo_rostros = nn.DataParallel(modelo_rostros)
modelo_rostros = modelo_rostros.to(dispositivo)

criterio = nn.BCEWithLogitsLoss()
parametros_entrenables = [p for p in modelo_rostros.parameters() if p.requires_grad]
optimizador = optim.Adam(parametros_entrenables, lr=mejores_params_transfer['tasa_aprendizaje'],
                      weight_decay=mejores_params_transfer['decaimiento_peso'])

num_epocas = 50
mejor_accuracy_val = 0.0
historial_transfer = {'perdida_entrenamiento': [], 'accuracy_entrenamiento': [], 
                     'perdida_validacion': [], 'accuracy_validacion': []}

for epoca in range(num_epocas):
    print(f"Epoca {epoca+1}/{num_epocas}")
    perdida_entrenamiento, accuracy_entrenamiento = entrenar_epoca(modelo_rostros, cargador_entrenamiento, criterio, optimizador, dispositivo, False)
    perdida_validacion, accuracy_validacion = validar_epoca(modelo_rostros, cargador_validacion, criterio, dispositivo, False)
    
    historial_transfer['perdida_entrenamiento'].append(perdida_entrenamiento)
    historial_transfer['accuracy_entrenamiento'].append(accuracy_entrenamiento)
    historial_transfer['perdida_validacion'].append(perdida_validacion)
    historial_transfer['accuracy_validacion'].append(accuracy_validacion)
    
    print(f"Entrenamiento: {perdida_entrenamiento:.4f}/{accuracy_entrenamiento:.4f}, Validacion: {perdida_validacion:.4f}/{accuracy_validacion:.4f}")
    
    if accuracy_validacion > mejor_accuracy_val:
        mejor_accuracy_val = accuracy_validacion
        modelo_guardar = modelo_rostros.module if usar_multi_gpu else modelo_rostros
        torch.save(modelo_guardar.state_dict(), Configuracion.DIR_MODELOS / "transfer_mejor.pth")

print(f"Transfer learning completado. Mejor Accuracy Val: {mejor_accuracy_val:.4f}")

Entrenando modelo de transfer learning final
Epoca 1/50


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home2/DSMaster/mgarcia/miniconda3/envs/rna_pytorch/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 85, in _worker
    output = module(*input, **kwargs)
  File "/home2/DSMaster/mgarcia/miniconda3/envs/rna_pytorch/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home2/DSMaster/mgarcia/miniconda3/envs/rna_pytorch/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/tmp/ipykernel_1194169/2043136025.py", line 46, in forward
    return self.clasificador(self.gap(self.caracteristicas(x))).squeeze(1)
  File "/home2/DSMaster/mgarcia/miniconda3/envs/rna_pytorch/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwa

In [ ]:
print("Optimizando hiperparametros de fine-tuning")
print(f"Trials: {Configuracion.TRIALS_FINETUNE}")

def objetivo_finetune(trial):
    tasa_aprendizaje = trial.suggest_loguniform('tasa_aprendizaje', 1e-6, 1e-3)
    decaimiento_peso = trial.suggest_loguniform('decaimiento_peso', 1e-7, 1e-4)
    num_capas_descongelar = trial.suggest_int('num_capas_descongelar', 4, 10)
    tamano_lote = trial.suggest_categorical('tamano_lote', [16, 32])
    
    transformacion_entrenamiento = obtener_transformaciones(Configuracion.TAMANO_IMG, aumentar=True)
    transformacion_validacion = obtener_transformaciones(Configuracion.TAMANO_IMG, aumentar=False)
    
    dataset_entrenamiento = SubconjuntoRostros(dataset_rostros_completo, indices_rostros_entrenamiento.indices, transformacion_entrenamiento)
    dataset_validacion = SubconjuntoRostros(dataset_rostros_completo, indices_rostros_validacion.indices, transformacion_validacion)
    
    cargador_entrenamiento = DataLoader(dataset_entrenamiento, batch_size=tamano_lote, shuffle=True,
                            num_workers=Configuracion.NUM_WORKERS, pin_memory=True)
    cargador_validacion = DataLoader(dataset_validacion, batch_size=tamano_lote, shuffle=False,
                          num_workers=Configuracion.NUM_WORKERS, pin_memory=True)
    
    modelo = ModeloReconocimientoFacial(
        modelo_base,
        fc_oculto=mejores_params_transfer['fc_oculto'],
        dropout=mejores_params_transfer['dropout'])
    
    modelo.descongelar_capas(num_capas=num_capas_descongelar)
    
    if usar_multi_gpu:
        modelo = nn.DataParallel(modelo)
    modelo = modelo.to(dispositivo)
    
    criterio = nn.BCEWithLogitsLoss()
    optimizador = optim.Adam(modelo.parameters(), lr=tasa_aprendizaje, weight_decay=decaimiento_peso)
    
    mejor_accuracy_val = 0.0
    for epoca in range(20):
        perdida_entrenamiento, accuracy_entrenamiento = entrenar_epoca(modelo, cargador_entrenamiento, criterio, optimizador, dispositivo, False)
        perdida_validacion, accuracy_validacion = validar_epoca(modelo, cargador_validacion, criterio, dispositivo, False)
        if accuracy_validacion > mejor_accuracy_val:
            mejor_accuracy_val = accuracy_validacion
        trial.report(accuracy_validacion, epoca)
        if trial.should_prune():
            raise optuna.TrialPruned()
    return mejor_accuracy_val

estudio_finetune = optuna.create_study(
    direction='maximize',
    study_name='fine_tuning',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=3))

estudio_finetune.optimize(objetivo_finetune, n_trials=Configuracion.TRIALS_FINETUNE, show_progress_bar=True)

print(f"Mejor accuracy: {estudio_finetune.best_value:.4f}")
print(f"Mejores parametros: {estudio_finetune.best_params}")

In [ ]:
print("Entrenando modelo final con fine-tuning")

mejores_params_finetune = estudio_finetune.best_params
transformacion_entrenamiento = obtener_transformaciones(Configuracion.TAMANO_IMG, aumentar=True)
transformacion_validacion = obtener_transformaciones(Configuracion.TAMANO_IMG, aumentar=False)

dataset_entrenamiento = SubconjuntoRostros(dataset_rostros_completo, indices_rostros_entrenamiento.indices, transformacion_entrenamiento)
dataset_validacion = SubconjuntoRostros(dataset_rostros_completo, indices_rostros_validacion.indices, transformacion_validacion)

cargador_entrenamiento = DataLoader(dataset_entrenamiento, batch_size=mejores_params_finetune['tamano_lote'],
                        shuffle=True, num_workers=Configuracion.NUM_WORKERS, pin_memory=True)
cargador_validacion = DataLoader(dataset_validacion, batch_size=mejores_params_finetune['tamano_lote'],
                      shuffle=False, num_workers=Configuracion.NUM_WORKERS, pin_memory=True)

modelo_final = ModeloReconocimientoFacial(
    modelo_base,
    fc_oculto=mejores_params_transfer['fc_oculto'],
    dropout=mejores_params_transfer['dropout'])

modelo_final.descongelar_capas(num_capas=mejores_params_finetune['num_capas_descongelar'])

if usar_multi_gpu:
    modelo_final = nn.DataParallel(modelo_final)
modelo_final = modelo_final.to(dispositivo)

criterio = nn.BCEWithLogitsLoss()
optimizador = optim.Adam(modelo_final.parameters(), lr=mejores_params_finetune['tasa_aprendizaje'],
                      weight_decay=mejores_params_finetune['decaimiento_peso'])

num_epocas = 30
mejor_accuracy_val = 0.0
historial_finetune = {'perdida_entrenamiento': [], 'accuracy_entrenamiento': [], 
                     'perdida_validacion': [], 'accuracy_validacion': []}

for epoca in range(num_epocas):
    print(f"Epoca {epoca+1}/{num_epocas}")
    perdida_entrenamiento, accuracy_entrenamiento = entrenar_epoca(modelo_final, cargador_entrenamiento, criterio, optimizador, dispositivo, False)
    perdida_validacion, accuracy_validacion = validar_epoca(modelo_final, cargador_validacion, criterio, dispositivo, False)
    
    historial_finetune['perdida_entrenamiento'].append(perdida_entrenamiento)
    historial_finetune['accuracy_entrenamiento'].append(accuracy_entrenamiento)
    historial_finetune['perdida_validacion'].append(perdida_validacion)
    historial_finetune['accuracy_validacion'].append(accuracy_validacion)
    
    print(f"Entrenamiento: {perdida_entrenamiento:.4f}/{accuracy_entrenamiento:.4f}, Validacion: {perdida_validacion:.4f}/{accuracy_validacion:.4f}")
    
    if accuracy_validacion > mejor_accuracy_val:
        mejor_accuracy_val = accuracy_validacion
        modelo_guardar = modelo_final.module if usar_multi_gpu else modelo_final
        torch.save(modelo_guardar.state_dict(), Configuracion.DIR_MODELOS / "final_mejor.pth")

modelo_guardar = modelo_final.module if usar_multi_gpu else modelo_final
torch.save(modelo_guardar.state_dict(), Configuracion.DIR_MODELOS / "modelo_final.pth")
print(f"Fine-tuning completado. Mejor Accuracy Val: {mejor_accuracy_val:.4f}")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

print("Evaluacion final")

modelo_final.eval()
todas_predicciones = []
todas_etiquetas = []
todas_probabilidades = []

with torch.no_grad():
    for entradas, etiquetas in cargador_validacion:
        entradas = entradas.to(dispositivo)
        salidas = modelo_final(entradas)
        probabilidades = torch.sigmoid(salidas)
        predicciones = (probabilidades > 0.5).float()
        todas_predicciones.extend(predicciones.cpu().numpy())
        todas_etiquetas.extend(etiquetas.numpy())
        todas_probabilidades.extend(probabilidades.cpu().numpy())

todas_predicciones = np.array(todas_predicciones)
todas_etiquetas = np.array(todas_etiquetas)
todas_probabilidades = np.array(todas_probabilidades)

metricas = {
    'accuracy': accuracy_score(todas_etiquetas, todas_predicciones),
    'precision': precision_score(todas_etiquetas, todas_predicciones),
    'recall': recall_score(todas_etiquetas, todas_predicciones),
    'f1_score': f1_score(todas_etiquetas, todas_predicciones),
    'auc_roc': roc_auc_score(todas_etiquetas, todas_probabilidades)}

matriz_confusion = confusion_matrix(todas_etiquetas, todas_predicciones)

print("Metricas finales:")
for k, v in metricas.items():
    print(f"{k}: {v:.4f}")

print("\nMatriz de confusion:")
print(matriz_confusion)

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(15, 12))

axes[0,0].plot(historial_preentrenamiento['perdida_entrenamiento'], label='Entrenamiento')
axes[0,0].plot(historial_preentrenamiento['perdida_validacion'], label='Validacion')
axes[0,0].set_title('Pre-entrenamiento - Perdida')
axes[0,0].legend()

axes[0,1].plot(historial_preentrenamiento['accuracy_entrenamiento'], label='Entrenamiento')
axes[0,1].plot(historial_preentrenamiento['accuracy_validacion'], label='Validacion')
axes[0,1].set_title('Pre-entrenamiento - Accuracy')
axes[0,1].legend()

axes[1,0].plot(historial_transfer['perdida_entrenamiento'], label='Entrenamiento')
axes[1,0].plot(historial_transfer['perdida_validacion'], label='Validacion')
axes[1,0].set_title('Transfer Learning - Perdida')
axes[1,0].legend()

axes[1,1].plot(historial_transfer['accuracy_entrenamiento'], label='Entrenamiento')
axes[1,1].plot(historial_transfer['accuracy_validacion'], label='Validacion')
axes[1,1].set_title('Transfer Learning - Accuracy')
axes[1,1].legend()

axes[2,0].plot(historial_finetune['perdida_entrenamiento'], label='Entrenamiento')
axes[2,0].plot(historial_finetune['perdida_validacion'], label='Validacion')
axes[2,0].set_title('Fine-tuning - Perdida')
axes[2,0].legend()

axes[2,1].plot(historial_finetune['accuracy_entrenamiento'], label='Entrenamiento')
axes[2,1].plot(historial_finetune['accuracy_validacion'], label='Validacion')
axes[2,1].set_title('Fine-tuning - Accuracy')
axes[2,1].legend()

plt.tight_layout()
plt.savefig(Configuracion.DIR_RESULTADOS / 'curvas_entrenamiento.png', dpi=300)
plt.show()

plt.figure(figsize=(8, 6))
sns.heatmap(matriz_confusion, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusion')
plt.ylabel('Real')
plt.xlabel('Predicho')
plt.savefig(Configuracion.DIR_RESULTADOS / 'matriz_confusion.png', dpi=300)
plt.show()

In [ ]:
resumen = f"""
Pre-entrenamiento:
  Trials: {Configuracion.TRIALS_PREENTRENAMIENTO}
  Mejores parametros: {estudio_preentrenamiento.best_params}
  Mejor accuracy validacion: {estudio_preentrenamiento.best_value:.4f}
  Accuracy validacion final: {historial_preentrenamiento['accuracy_validacion'][-1]:.4f}

Transfer Learning:
  Trials: {Configuracion.TRIALS_TRANSFER}
  Mejores parametros: {estudio_transfer.best_params}
  Mejor accuracy validacion: {estudio_transfer.best_value:.4f}
  Accuracy validacion final: {historial_transfer['accuracy_validacion'][-1]:.4f}

Fine-tuning:
  Trials: {Configuracion.TRIALS_FINETUNE}
  Mejores parametros: {estudio_finetune.best_params}
  Mejor accuracy validacion: {estudio_finetune.best_value:.4f}
  Accuracy validacion final: {historial_finetune['accuracy_validacion'][-1]:.4f}

Metricas Finales:
  Accuracy: {metricas['accuracy']:.4f}
  Precision: {metricas['precision']:.4f}
  Recall: {metricas['recall']:.4f}
  F1-Score: {metricas['f1_score']:.4f}
  AUC-ROC: {metricas['auc_roc']:.4f}
"""

print(resumen)

with open(Configuracion.DIR_RESULTADOS / 'resumen.txt', 'w') as f:
    f.write(resumen)

print("\nEntrenamiento completado")